In [1]:
from glob import glob
combo_files = list(glob('data/processed/*/*combo*.csv'))
print(combo_files)

['data/processed/01_the_colour_of_magic/01_the_colour_of_magic_combos.csv', 'data/processed/02_the_light_fantastic/02_the_light_fantastic_combos.csv', 'data/processed/03_equal_rites/03_equal_rites_combos.csv', 'data/processed/04_mort/04_mort_combos.csv', 'data/processed/05_sourcery/05_sourcery_combos.csv', 'data/processed/06_wyrd_sisters/06_wyrd_sisters_combos.csv', 'data/processed/07_pyramids/07_pyramids_combos.csv', 'data/processed/08_guards!_guards!/08_guards!_guards!_combos.csv', 'data/processed/09_eric/09_eric_combos.csv', 'data/processed/10_moving_pictures/10_moving_pictures_combos.csv', 'data/processed/11_reaper_man/11_reaper_man_combos.csv', 'data/processed/12_witches_abroad/12_witches_abroad_combos.csv', 'data/processed/13_small_gods/13_small_gods_combos.csv', 'data/processed/14_lords_and_ladies/14_lords_and_ladies_combos.csv', 'data/processed/15_men_at_arms/15_men_at_arms_combos.csv', 'data/processed/24_fifth_elephant/24_fifth_elephant_combos.csv', 'data/processed/25_the_trut

In [2]:
import pandas as pd
from pathlib import Path
dataframes = [pd.read_csv(f) for f in combo_files]
for f, df in zip(combo_files,dataframes): df['book'] = Path(f).stem

#display(dataframes[0])
full_df = pd.concat(dataframes)
full_df.fillna(0, inplace= True)
#display(full_df)
#display(full_df.sum())

In [4]:
import re
display(full_df.columns[1:] != 'book')


sum_series = full_df.loc[:,[False, *(full_df.columns[1:] != 'book')]].sum()


filtered = sum_series[sum_series > 1]

new_cols = []
for c in filtered.index:
    c = c.replace('"Great A\'Tuin "',"'Great ATuin'")
    c = c.replace('"Awf\'ly Wee Billy "',"'Awfly Wee Billy'")    
    
    match = re.match(r".*'([^']+)'.*,.*'([^']+)'",c)
    if not match: 
        print(f"Error: {c}")
        new_cols.append(c)
        continue
    #print('match!')
    matches = match.group(1,2)
    new_cols.append(( matches[0].strip(), matches[1].strip()))
filtered.index = new_cols

import networkx as nx
from itertools import chain

G = nx.Graph()

nodes = set(chain(*[[*c] for c in filtered.index]))
G.add_nodes_from(nodes)

edges_with_weights = [(*edge,weight) for edge, weight  in zip(filtered.index, filtered.values)]

G.add_weighted_edges_from(edges_with_weights)
nx.write_gexf(G, f"full_graph.gexf")

array([ True,  True,  True, ...,  True,  True,  True])